In [ ]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 13.1 MB/s eta 0:00:00


In [ ]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Check if GPU is available and set device accordingly
print("device : ", device)

device :  cuda


In [ ]:
file_path = "/content/gdrive/My Drive/Dataset.csv"  
data = pd.read_csv(file_path, sep='\t')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15824 entries, 0 to 15823
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   content  15824 non-null  object 
 1   lable    15515 non-null  float64
dtypes: float64(1), object(1)
memory usage: 247.4+ KB


In [ ]:
null = data[data.lable.isnull()].index
data.loc[null,"content"]

1602     응애 응애 엄마 저 맘에 안들죠? ........아들 ?? " 너 내가 우스워 보이...
1654            토니스타크 평소 "아이엠그루트"라는 유행어를 부러워했다는게 학계의 정설\t1
1992     "13일 현대차에 따르면 올 들어 국내 소비자들의 수입차 구매의향률이 3년 만에 하...
2920                  에이프릴이 한마디 합니다 "예쁜게 죄" 구하라님 "무기징역"\t1
3720           답글 글씨체를 봐라 저게 애새끼가 쓴거냐?"빨갱이새끼가 쓴거지 ㅁㅈㅎㅉㅉ\t0
                               ...                        
15754                         저거 직진 , 후진 가능하다는 표시 아니였음?\t1
15797    손주은 센세가 떠오르노 ㅋㅋㅋㅋㅋㅋㅋ강의시간에 너 공부 안하면 창녀보다 못한 삶을 ...
15805                뭘 모르나 본데, 머리 길면 여군. 빡빡이는 군인. 알간???\t1
15808              1. 한국전쟁 당시, 미국 외교 담당 부서인 CFR은 김일성에게 \t1
15820    세금 내놓으라고 데모질 중 ㅋㅋ간첩, 도둑놈 새끼들이 대통령 해처먹으니까  나도 같...
Name: content, Length: 309, dtype: object

In [ ]:
data.loc[null,"lable"] = data.loc[null, "content"].apply(lambda x:x[-1])
data.loc[null,"content"] = data.loc[null, "content"].apply(lambda x:x[-2])

In [ ]:
data = data.astype({"lable":"int"})
data.info()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/core/generic.py:6226 in astype                    │
│                                                                                                  │
│    6223 │   │   │   │   if isna(cdt):                                                            │
│    6224 │   │   │   │   │   res_col = col.copy() if copy else col                                │
│    6225 │   │   │   │   else:                                                                    │
│ ❱  6226 │   │   │   │   │   res_col = col.astype(dtype=cdt, copy=copy, errors=errors)            │
│    6227 │   │   │   │   results.append(res_col)                                                  │
│    6228 │   │                                                                                    │
│    6229 │   │   elif is_extension_array_dtype(dtype) and self.ndim > 1:                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/core/generic.py:6240 in astype                    │
│                                                                                                  │
│    6237 │   │                                                                                    │
│    6238 │   │   else:                                                                            │
│    6239 │   │   │   # else, only a single dtype is given                                         │
│ ❱  6240 │   │   │   new_data = self._mgr.astype(dtype=dtype, copy=copy, errors=errors)           │
│    6241 │   │   │   return self._constructor(new_data).__finalize__(self, method="astype")       │
│    6242 │   │                                                                                    │
│    6243 │   │   # GH 33113: handle empty frame or series                                         │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/core/internals/managers.py:448 in astype          │
│                                                                                                  │
│    445 │   │   )                                                                                 │
│    446 │                                                                                         │
│    447 │   def astype(self: T, dtype, copy: bool = False, errors: str = "raise") -> T:           │
│ ❱  448 │   │   return self.apply("astype", dtype=dtype, copy=copy, errors=errors)                │
│    449 │                                                                                         │
│    450 │   def convert(                                                                          │
│    451 │   │   self: T,                                                                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/core/internals/managers.py:352 in apply           │
│                                                                                                  │
│    349 │   │   │   │   if callable(f):                                                           │
│    350 │   │   │   │   │   applied = b.apply(f, **kwargs)                                        │
│    351 │   │   │   │   else:                                                                     │
│ ❱  352 │   │   │   │   │   applied = getattr(b, f)(**kwargs)                                     │
│    353 │   │   │   except (TypeError, NotImplementedError):

In [ ]:
# 전체 데이터를 학습+검증 데이터셋과 테스트 데이터셋으로 분할
train_val_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# 학습+검증 데이터셋을 학습 데이터셋과 검증 데이터셋으로 분할
train_data, val_data = train_test_split(train_val_data, test_size=0.25, random_state=42)

In [ ]:
MODEL_NAME = "beomi/kcbert-base"
  
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
tokenized_train_sentences = tokenizer(
    list(train_data["content"]),
    return_tensors="pt",
    max_length = 128,
    padding = True,
    truncation = True,
    add_special_tokens = True,
)

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
tokenized_val_sentences = tokenizer(
    list(val_data["content"]),
    return_tensors="pt",
    max_length = 128,
    padding = True,
    truncation = True,
    add_special_tokens = True,
)

In [ ]:
tokenized_test_sentences = tokenizer(
    list(train_data["content"]),
    return_tensors="pt",
    max_length = 128,
    padding = True,
    truncation = True,
    add_special_tokens = True,
)

In [ ]:
train_label = train_data["lable"].values
val_label = val_data["lable"].values
test_label = test_data["lable"].values

# 각 데이터셋에 대한 CurseDataset 객체 생성
train_dataset = CustomDataset(tokenized_train_sentences, train_label)
val_dataset = CustomDataset(tokenized_val_sentences, val_label)
test_dataset = CustomDataset(tokenized_test_sentences, test_label)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)

In [ ]:
training_args = TrainingArguments(
    output_dir = './results',          # output directory
    num_train_epochs = 10,             # total number of training epochs
    per_device_train_batch_size = 8,  # batch size per device during training
    per_device_eval_batch_size = 64,   # batch size for evaluation
    logging_dir = './logs',            # directory for storing logs
    logging_steps = 500,
    save_total_limit = 2,  # 체크포인트 파일의 최대 개수
)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
trainer = Trainer(
    model = model,                         
    args = training_args,                  
    train_dataset = train_dataset,         
    eval_dataset = val_dataset,  
    compute_metrics = compute_metrics,       
)

In [ ]:
# 모델 학습 및 검증
trainer.train()

In [ ]:
trainer.evaluate(eval_dataset = test_dataset)

In [ ]:
def sentence_predict(sent):
  model.eval()

  tokenized_sent = tokenizer(
    sent,
    return_tensors="pt",
    max_length = 128,
    truncation = True,
    add_special_tokens = True,
  )

  # Move input tensors to the same device as the model
  tokenized_sent.to(device)

  with torch.no_grad():
    outputs = model(
        input_ids = tokenized_sent["input_ids"],
        attention_mask = tokenized_sent["attention_mask"],
        token_type_ids = tokenized_sent["token_type_ids"],
        )

    logits = outputs[0]
    logits = logits.detach().cpu()  # Move logits back to CPU for further processing
    result = logits.argmax(-1)
    if result == 0:
      result = ">> 악성댓글 😈😈"
    elif result == 1:
      result = ">> 정상댓글 😁😁"
    return result

while True:
  sentence = input("댓글을 입력하세요 :")
  if sentence == "0":
    break
  print(sentence_predict(sentence))
  print("\n")